In [1]:
# To default to float division and print function.
from __future__ import (division, print_function)

# Core python libraries
import csv
import re
import math

# External libraries.
import nltk
import numpy
import pandas
import scipy
import sklearn
# Sklearn is not eager in importing its dependancies. Have to explicity load them.
import sklearn.ensemble

# So you know when this code block finishes.
print ("Done")

Done


In [2]:
data = pandas.read_csv('data/train_medium.csv')

X_train_index, X_test_index, Y_train, Y_test = sklearn.cross_validation.train_test_split(
    data.index, data['LogSalaryNormalized'], test_size=.33, random_state=42)

# Keep train and test as pandas dataframes.
X_train = data.iloc[X_train_index]
X_test = data.iloc[X_test_index]

data.head()


,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName,DescriptionLength,LogSalaryNormalized
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,348,10.126631
1,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk,397,10.221941
2,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,216,10.126631
3,13179816,Engineering Systems Analyst Water Industry,Engineering Systems Analyst Water Industry Loc...,"Dorking, Surrey, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20K to 30K,25000,cv-library.co.uk,1602,10.126631
4,14131336,Senior Subsea Pipeline Integrity Engineer,A globally renowned engineering and training c...,"Aberdeen, Borders",UK,NaN,permanent,Indigo 21 Ltd,Engineering Jobs,50000 - 100000/annum,75000,cv-library.co.uk,1364,11.225243


In [3]:
MIN_WORD_FREQUENCY = 5

count_vect = sklearn.feature_extraction.text.CountVectorizer(
  stop_words='english', min_df=MIN_WORD_FREQUENCY)

# Here I am count vectoring the full description field. But in theory
# any text field can be processed this way.
X_train_full_description_counts = count_vect.fit_transform(X_train['FullDescription'])
X_test_full_description_counts = count_vect.transform(X_test['FullDescription'])

print(X_train_full_description_counts.shape)
print(X_test_full_description_counts.shape)
#TODO (max): supress the VisibleDeprecationWarning here from numpy. It is being thrown by the package.

(670, 1789)
(330, 1789)


In [6]:
def original_salary_scale(log_value):
    return math.exp(log_value)

def mean_absolute_error_salary_scale(y_test, y_predicted):
    return sklearn.metrics.mean_absolute_error(
        numpy.exp(y_test), numpy.exp(y_predicted))

# Guess the average. Create an empty vector of the desired shape.
average_guess = numpy.empty(Y_test.shape)
average_guess.fill(numpy.mean(Y_train))

mean_guess = average_guess[0]
print('Mean salary value in training set is  £{:10.2f}'.format(original_salary_scale(mean_guess)))
average_guess_mae = mean_absolute_error_salary_scale(Y_test, average_guess)
print('Guess the average Mean Absolute Error: {:10.4f}'.format(average_guess_mae))

# SGD Needs normalized inputs
# TODO: Suppress DataConversionWarning from converting ints to floats here.
normalizer = sklearn.preprocessing.Normalizer(norm='l1')
X_train_norm = normalizer.fit_transform(X_train_full_description_counts.astype('float64'))
X_test_norm = normalizer.transform(X_test_full_description_counts.astype('float64'))

# We want a stochastic gradient descent with l1 norm.
sgd = sklearn.linear_model.SGDRegressor(alpha=.005, penalty='l1', n_iter=10000)
sgd.fit(X_train_norm, Y_train)
sgd_predictions = sgd.predict(X_test_norm)
sgd_mae = mean_absolute_error_salary_scale(Y_test, sgd_predictions)
print('SGDRegressor Mean Absolute Error: {:10.4f}'.format(sgd_mae))
#TODO (any): wonder why this is so inaccurate/wrong

# Random Forest.
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=10)
rf.fit(X_train_full_description_counts, Y_train)
rf_predictions = rf.predict(X_test_full_description_counts)

# Mean Absolute Error
rf_mae = mean_absolute_error_salary_scale(Y_test, rf_predictions)
print('Random Forest Regressor Mean Absolute Error: {:10.4f}'.format(rf_mae))

Mean salary value in training set is  £  25941.22
Guess the average Mean Absolute Error:  9396.2364
SGDRegressor Mean Absolute Error:  9306.9440
Random Forest Regressor Mean Absolute Error:  5254.7427


In [8]:
print(sgd.intercept_)
print(mean_guess)

[ 9.94625975]
10.1635886033
